<a href="https://colab.research.google.com/github/DsStudentSid/DL-Assignments/blob/master/Melanoma_Tumor_Size.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Table of Content**

To train a model using TF-Keras library and predict the melanoma tumor size

**Melanoma Tumor Size Dataset **

2.1 Get the Data

2.2 Clean the Data

2.3 Split the Data into Train and Test

2.4 Inspect the Data

2.5 Split Features from Labels

2.6 Normalize the Data

**The Model**

3.1 Build the Model

3.2 Inspect the Model

3.3 Train the Model

3.4 Make Predictions

**Model Tuning**

4.1 Increasing the Number of Hidden Layers and Hidden Units

4.2 Weight Regularization

4.3 Dropout

4.4 Combined L2 + Dropout

4.5 Batch Normalization

**Conclusion**

In [ ]:
# Import tensorflow 2.x
# This code block will only work in Google Colab.
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.3.0


In [2]:
!pip install pandas_profiling==2.5.0

In [3]:
import pathlib

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns


In [4]:
df_melona = pd.read_csv('https://raw.githubusercontent.com/DsStudentSid/DL-Assignments/master/melanoma_tumor_size.csv')
df_melona.head()

,mass_npea,size_npear,malign_ratio,damage_size,exposed_area,std_dev_malign,err_malign,malign_penalty,damage_ratio,tumor_size
0,6930.90,2919.02,0.42116,51.8298,9.888294e+05,109.487,2758.76,72,39.3620,14.103
1,15635.70,4879.36,0.31206,223.5500,2.058426e+06,248.881,5952.53,240,22.0253,2.648
2,10376.20,2613.88,0.25191,127.3370,1.434676e+06,160.093,4635.26,73,29.9963,1.688
3,13093.80,4510.06,0.34444,155.4400,1.812195e+06,173.015,5273.87,32,28.1354,3.796
4,7545.21,2882.36,0.38201,85.1237,1.043918e+06,124.414,3263.35,57,35.0200,18.023
